In [1]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [2]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [3]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_perf_ratios.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-07
DEST_DIR: ..\data

source_path: ..\data\2025-04-07_df_OHLCV_clean.parquet
dest_path: ..\data\2025-04-07_df_perf_ratios.parquet


In [4]:
import pandas as pd

# Get tickers from df_finviz.pkl file
df_finviz = pd.read_parquet(f'..\data\{date_str}_df_finviz.parquet')
tickers = df_finviz.index.to_list()

In [5]:
import pandas as pd

pd.set_option('display.width', 1000) 

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# # Display the first few rows of the DataFrame to verify
# df = df[['Adj Close']].copy()
display(df)

Open    High     Low   Close  Adj Close   Volume    Adj Open    Adj High     Adj Low
Symbol Date                                                                                              
SCHC   2025-04-07   32.15   33.41   30.84   32.41      32.41  1831992   32.150000   33.410000   30.840000
       2025-04-04   34.10   34.15   33.04   33.15      33.15   820300   34.100000   34.150000   33.040000
       2025-04-03   35.81   36.03   35.39   35.42      35.42   288400   35.810000   36.030000   35.390000
       2025-04-02   35.64   36.20   35.64   36.08      36.08   586300   35.640000   36.200000   35.640000
       2025-04-01   35.92   36.10   35.74   35.96      35.96   701800   35.920000   36.100000   35.740000
...                   ...     ...     ...     ...        ...      ...         ...         ...         ...
WAB    2024-04-12  146.53  147.85  145.95  146.18     145.48   848000  145.828324  147.142003  145.251101
       2024-04-11  147.47  148.53  146.65  147.71     147.01  1262500  146.771137  147.826114  145.955023
       2024-04-10  146.10  148.34  145.16  147.54     146.84  1512500  145.406832  147.636204  144.471292
       2024-04-09  149.45  149.93  146.96  147.86     147.16   816700  148.742473  149.220200  146.264261
       2024-04-08  150.50  150.61  149.33  149.46     148.75  1002900  149.785060  149.894537  148.620618

[385536 rows x 9 columns]

In [6]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
SCHC   2025-04-07      32.41
       2025-04-04      33.15
       2025-04-03      35.42
       2025-04-02      36.08
       2025-04-01      35.96
...                      ...
WAB    2024-04-12     145.48
       2024-04-11     147.01
       2024-04-10     146.84
       2024-04-09     147.16
       2024-04-08     148.75

[385536 rows x 1 columns]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 385536 entries, ('SCHC', Timestamp('2025-04-07 00:00:00')) to ('WAB', Timestamp('2024-04-08 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  385536 non-null  float64
dtypes: float64(1)
memory usage: 4.5+ MB


In [8]:
import pandas as pd

def select_tickers_data(df, tickers):
  """
  Selects data for a list of tickers from a DataFrame with a MultiIndex
  where the first level is the ticker and the second level is the date.

  Args:
    df (pd.DataFrame): The input DataFrame with a MultiIndex.
    tickers (list): A list of ticker symbols to select.

  Returns:
    pd.DataFrame: A DataFrame containing only the data for the specified tickers.
                  Rows for tickers not found will not be included.
  """
  valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

  if not valid_tickers:
    print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
    return pd.DataFrame()

  try:
    ticker_df = df.loc[valid_tickers]
    return ticker_df
  except KeyError as e:
    print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
    return pd.DataFrame()


# Example usage:
# ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
selected_data = select_tickers_data(df, tickers)
print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-04-07     181.46
       2025-04-04     188.38
       2025-04-03     203.19
       2025-04-02     223.89
       2025-04-01     223.19
...                      ...
IBDS   2024-04-12      22.55
       2024-04-11      22.51
       2024-04-10      22.49
       2024-04-09      22.65
       2024-04-08      22.62

[385536 rows x 1 columns]


In [9]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [10]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
      Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A     -9.346100  -10.199732  0.091336  -9.201534   -9.034785  0.193912   -9.726435    -8.828324   0.144596   -8.486498    -7.902944   0.185790   -6.116972    -6.281224   0.329032   -3.825081    -4.416985   0.511611    -2.450182     -3.047658    0.655733    -1.274717     -1.639429    0.799936
AA   -13.656439  -12.263485  0.000000 -11.642557  -10.384648  0.107104  -12.104034   -10.086776   0.070707   -8.635567    -8.112576   0.182690   -4.499631    -4.952086   0.462180   -2.775692    -3.226108   0.622758    -2.142211     -2.611681    0.690291    -0.660505     -0.894123    0.896272
AAL    7.387256   43.214103  4.849818  -3.457470   -4.135506  0.504518   -7.4386

In [11]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sortino 3d
  Inf: ['BILS', 'DECK', 'DLTR', 'FND', 'GME', 'LULU', 'ONON', 'SAIA', 'SN', 'WSM']
Column: Omega 3d
  NaN: ['BILS', 'DECK', 'DLTR', 'FND', 'GME', 'LULU', 'ONON', 'SAIA', 'SN', 'WSM']
df_to_save has_nan_inf: False
NaN/Inf Locations AFTER Replacement:
No NaN or Inf values found.
      Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A     -9.346100  -10.199732  0.091336  -9.201534   -9.034785  0.193912   -9.726435    -8.828324   0.144596   -8.486498    -7.902944   0.185790   -6.116972    -6.281224   0.329032   -3.825081    -4.416985   0.511611    -2.450182     -3.047658    0.655733    -1.274717     -1.639429 

In [12]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'BILS'

#### Use Cell below to check the inf, -inf and NaN replacement

In [13]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       14.402118
Sortino 3d            inf
Omega 3d              NaN
Sharpe 5d        4.289810
Sortino 5d       8.332093
Omega 5d         2.049745
Sharpe 10d       4.838264
Sortino 10d      9.487835
Omega 10d        2.300684
Sharpe 15d       2.888126
Sortino 15d      5.230933
Omega 15d        1.617945
Sharpe 30d       1.821633
Sortino 30d      3.146058
Omega 30d        1.341344
Sharpe 60d       1.715071
Sortino 60d      2.916696
Omega 60d        1.306241
Sharpe 120d      2.089566
Sortino 120d     3.549372
Omega 120d       1.393452
Sharpe 250d      3.545578
Sortino 250d     6.839078
Omega 250d       1.786726
Name: BILS, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d        14.402118
Sortino 3d      513.959059
Omega 3d         46.787112
Sharpe 5d         4.289810
Sortino 5d        8.332093
Omega 5d          2.049745
Sharpe 10d        4.838264
Sortino 10d       9.487835
Omega 10d         2.300684
Sharpe 15d        2.888126
Sortino 15d       5.230933
Omega 15d         1.617945
Sharpe 30d        1.821633
Sortino 30d       3.146058
Omega 30d         1.341344
Sharpe 60d        1.715071
Sortino 60d       2.916696
Omega 60d         1.306241
Sharpe 120d       2.089566
Sortino 120d      3.549372
Omega 120d        1.393452
Sharpe 250d       3.545578
Sortino 250d      6.839078
Omega 250d        1.786726
Name: BILS, dtype: float64

In [14]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path)
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-04-07_df_perf_ratios.parquet


In [15]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['USFR','AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'VCIT']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====